In [2]:
#for making alterations to TF binding motifs after getting sequences in SuPreMo

In [3]:
%load_ext autoreload
%autoreload 2

In [21]:
import basenji
import pandas as pd 
import numpy as np
import os, psutil, io, gzip, time 
import pysam
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
from pybedtools import BedTool

In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

In [6]:
hg38_file = '/home/data/genomes/hg38.fa'
hg38_fa = pysam.Fastafile(hg38_file)

In [11]:
colnames = ['CHROM', 'POS', 'END', 'REF', 'ALT', 'SVTYPE', 'SVLEN']

test_dir='/home/shu_zhang/SuPreMo/test_data/test_set_edge_cases/'
test_sv=pd.read_csv(f'{test_dir}test_set_edge_SV.bed', sep='\t', names=colnames)
test_sv_no_bnd=test_sv[test_sv.SVTYPE!='BND']
test_simple=pd.read_csv(f'{test_dir}test_set_edge_simple.bed', sep='\t', names=colnames)


In [12]:
variants=test_sv_no_bnd
variants

,CHROM,POS,END,REF,ALT,SVTYPE,SVLEN
0,chr1,244397848,244398348.0,AAACTAAACTAAAATAAACTAAAATAAACTAAAATAAACTAAACTA...,A,DEL,500.0
1,chr1,244397848,244398348.0,A,<DEL>,DEL,500.0
2,chr1,244397848,244398348.0,A,<DUP:TANDEM>,DUP,500.0
3,chr1,244397848,244398348.0,A,<INV>,INV,500.0
4,chr1,244397848,244402848.0,A,<DEL>,DEL,5000.0
5,chr1,244397848,244402848.0,A,<DUP:TANDEM>,DUP,5000.0
6,chr1,244397848,244402848.0,A,<INV>,INV,5000.0
7,chr2,400000,400500.0,CTATGGAACAGGCTGGGGTTACTTGGCTATAGAACAGGCCGGGGCC...,C,DEL,500.0
8,chr2,400000,400500.0,C,<DEL>,DEL,500.0
9,chr2,400000,400500.0,C,<DUP:TANDEM>,DUP,500.0


In [20]:
#read in CTCF overlap file
ctcf_track=pd.read_csv('/home/shu_zhang/SuPreMo/mutate_data/MA0139.2.tsv.gz', sep='\t', names=['chr', 'start', 'end', 'tf', 'idk1' ,'idk2', 'idk3'])
#just look at the one for the first few variants in supremo 
#subset to within 1MB
#these variants are in normal range, i think
#ctcf_chr1=ctcf_track[(ctcf_track.chr=='chr1') & (ctcf_track.start>243397848) & (ctcf_track.end<245397848)]

flank_len=5000
ctcf_chr1=ctcf_track[(ctcf_track.chr=='chr1') & (ctcf_track.start>(244397848-flank_len)) & (ctcf_track.end<(244398348.0+flank_len))]

ctcf_chr1

,chr,start,end,tf,idk1,idk2,idk3
174915,chr1,244394527,244394542,CTCF,851,429,-
174916,chr1,244395201,244395216,CTCF,818,372,+
174917,chr1,244395523,244395538,CTCF,826,386,+
174918,chr1,244397734,244397749,CTCF,817,371,+
174919,chr1,244398211,244398226,CTCF,828,388,-
174920,chr1,244401218,244401233,CTCF,833,397,+


In [ ]:
    aludf=pd.concat([data_df, score_df], axis=1)
    aludf['POS']=aludf['POS'].astype(int)
    aludf['END']=aludf['END'].astype(int)
    alu_colorder = ['CHROM', 'POS', 'END']

In [ ]:
#what we get out of supremo:
#sequences
#var_rel_pos --> gives start of variant for REF_ALT, based on position from 0 (start of sequence)

In [ ]:
#track code 


        
#clear files from previous tracks 
pybedtools.cleanup()
        
        #this is not ideal bc cleanup is removing it everytime, so need to keep loading it in
        top_bedtool_unsort=BedTool.from_dataframe(top_10[alu_colorder])
        top_bedtool=BedTool.sort(top_bedtool_unsort)
        bottom_bedtool_unsort=BedTool.from_dataframe(bottom_10[alu_colorder])
        bottom_bedtool=BedTool.sort(bottom_bedtool_unsort)
        
        try:
            track_df=pd.read_csv(f'{tf_tracks}{track_id}', sep='\t', header=None, skiprows=1,
                            names=['chr', 'start', 'end', 'tf', 'rel_score', '-log10(pval)', 'strand'])
            #print(track_df)
            #print((track_df.columns))
            tf_name=track_df.iloc[1,3]
            print(tf_name)
        
            track_df['start']=track_df['start'].astype(int)
            track_df['end']=track_df['end'].astype(int)
            track_bedtool=BedTool.from_dataframe(track_df) 
        
            #top and bottom intersections 
            top_intersect=top_bedtool.intersect(track_bedtool, c=True, F=0.5).to_dataframe(header=None)
            bottom_intersect=bottom_bedtool.intersect(track_bedtool, c=True, F=0.5).to_dataframe(header=None)
    

        except:
            print(f'error in index {counter}')

In [ ]:
def mutate_TFs(POS, END, SVTYPE, SVLEN, sequences_i, up_flank, down_flank, revcomp, alter_variant, track_path):

    """
    This function mutates every TF within the variant and its flanking region 

    Note to self: use a different function for shuffling motifs, that one is easier / doesn't need to count up to 1024s
    
    Arguments:
        up_flank: 
            - length of bp upstream of variant to look for TF motifs 
        down_flank:
            - length of bp downstream of variant to look for TF motifs         
        alter_variant: 
            - if True, we use the ALT sequence (variant included)
            - if False, we use the REF sequence (variant not included) 
        track_path: 
            - path to TF binding track. For now, must be JASPAR-style bed tracks 

    Returns: 
        sequences_i: tuple containing:
            1. REF sequence (index 0)
            2. REF-altered sequence based on the TF motifs (index 1)
            3. variant POS indices relative to start for [REF, ALT] (index 2)

    
        Note to self: this is different for BNDs!!!!!!!!!!!!!!!!


    """

    

    if SVTYPE=='BND':
        #FIX THIS LATER 
        return(sequences_i)
        
    
    if variant_start < 0 or variant_end >= len(seq) or variant_start > variant_end:
        raise ValueError("Invalid range of indices")
        
           
    if (variant_start-posflank)<0 or (variant_end+endflank)>len(seq):
        raise ValueError("Flanking indices exceed available sequence length")
    
    mutated_sequence = list(seq)
    if posflank>0:
        region_start=variant_start-posflank
        region_end=variant_start
    if endflank>0:
        region_start=variant_end
        region_end=variant_end+endflank
        
    #right now, just assumes that revcomp does not deal with flanking sequences 
    #will need to fix it later
    if revcomp:
        #also make sure these are not one off 
        mut_start=len(seq)-variant_end
        mut_end=len(seq)-variant_start
    
    else:
        shuff_start=variant_start
        shuff_end=variant_end
          
        
    seq_list = list(seq)
    seq_to_shuffle = seq_list[shuff_start:shuff_end + 1]
    random.shuffle(seq_to_shuffle)
    seq_list[shuff_start:shuff_end + 1] = seq_to_shuffle

    return ''.join(seq_list)  

In [ ]:
#NOTE TO SELF: NEED TO GET LIST/TUPLE OF MOTIF COORDINATES AND REMOVE THEM

def remove_substrings(string, coordinates):
    result = ""
    index = 0
    
    for start, end in coordinates:
        result += string[index:start]
        index = end + 1
    
    result += string[index:]
    
    return result

# Example usage:
string = "abcdefgh"
coordinates_to_remove = [(1, 3), (5, 6)]
result = remove_substrings(string, coordinates_to_remove)
print(result)  #